## Import packages

In [1]:
%matplotlib inline
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from CFModel import CFModel

c:\users\ayrton\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ayrton\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ayrton\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ayrton\

## Define constants

In [2]:
RATINGS_CSV_FILE = './input/training.txt'
MODEL_WEIGHTS_FILE = 'movie_weights.h5'
K_FACTORS = 120
RNG_SEED = 144653

## Load Recipe data

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep=',', 
                      encoding='utf-8', 
                      names=['userid','movieid','rating','timestamp'])
max_userid = ratings['userid'].max() + 1
max_movieid = ratings['movieid'].max() + 1
print(len(ratings), 'ratings loaded.')

700146 ratings loaded.


In [4]:
ratings.head()

,userid,movieid,rating,timestamp
0,63,3006,4,977899676
1,1125,10,2,975625890
2,181,2384,2,977088920
3,1968,1610,4,974742431
4,4156,2410,4,965342309


## Create training set

In [8]:
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)
Users = shuffled_ratings['userid'].values
print('Users:', Users, ', shape =', Users.shape)
Movie = shuffled_ratings['movieid'].values
print('Movies:', Movie, ', shape =', Movie.shape)
Ratings = shuffled_ratings['rating'].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [5289 4979 5449 ... 5779 5949  349] , shape = (700146,)
Movies: [ 902 1129  349 ... 1711  434  380] , shape = (700146,)
Ratings: [4 3 5 ... 3 1 3] , shape = (700146,)


## Define model

In [10]:
model = CFModel(max_userid, max_movieid, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

D:\Code\Semester 7\Data Science\Recomendation\collaborative_filtering_keras\CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1))


## Train model

In [12]:
callbacks = [EarlyStopping('val_loss', patience=2),
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = model.fit([Users, Movie], Ratings, nb_epoch=30, validation_split=.1, verbose=1, callbacks=callbacks)

c:\users\ayrton\appdata\local\programs\python\python35\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 630131 samples, validate on 70015 samples
Epoch 1/30
358560/630131 [================>.............] - ETA: 1:24 - loss: 12.1708

KeyboardInterrupt: 

## Plot training and validation RMSE

In [21]:
loss = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                     'training': [ math.sqrt(loss) for loss in history.history['loss'] ],
                     'validation': [ math.sqrt(loss) for loss in history.history['val_loss'] ]})
#ax = loss.ix[:,:].plot(x='epoch', figsize={1,30}, grid=True)
ax = loss.ix[:,:].plot(x='epoch', grid=True)
ax.set_ylabel("root mean squared error")
ax.set_ylim([-2.0,5.0]);

NameError: name 'history' is not defined

In [22]:
trained_model = CFModel(max_userid, max_movieid, K_FACTORS)

D:\Code\Semester 7\Data Science\Recomendation\collaborative_filtering_keras\CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1))


In [23]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [24]:
def predict_rating(userid, movieid):
    return trained_model.rate(userid, movieid)

In [11]:
test = pd.read_csv('input' + '/testing.txt', delimiter=',', 
                    names=['user_id', 'movie_id','timestamp'])

In [13]:
ids, predicted = [], []
for i in range(test.shape[0]):
    ids.append(i+1)
    predicted.append(predict_rating(test.iloc[i][0],test.iloc[i][1]))


In [14]:
submission = pd.DataFrame({
    'ID': ids,
    'Predicted': predicted
})

In [ ]:
submission.to_csv('I-CFN.csv', index=False)